In [1]:
import mysql.connector
from mysql.connector import Error

In [2]:
def create_server_connection(host_name, user_name, user_password):

    connection = None

    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            auth_plugin="mysql_native_password",
            buffered=True,
        )

        print("MySQL Database connection successful")
        return connection

    except Error as err:
        print(f"Error: {err}")


pw = "qwerasdf1@"
connection = create_server_connection("localhost", "root", pw)

MySQL Database connection successful


In [4]:
# # Execute sql query
# def execute_query(connection, query):
#     """
#     exqcute a specific query
#     """
#     cursor = connection.cursor()
#     try:
#         cursor.execute(query)
#         connection.commit()
#         print("Query was successful")

#     except Error as err:
#         print(f"Error: '{err}'")


# # execute_query(connection, 'CREATE DATABASE qa_database;')
# # execute_query(connection, 'USE qa_database;')

Query was successful
Query was successful


In [20]:
def execute_query(connection, query):
    """
    return all results from a queries
    """
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        return results

    except Error as err:
        print(f"Error: '{err}'")


query = """

CREATE DATABASE qa_database;
USE qa_database;

"""
execute_query(connection, query)

Error: '1007 (HY000): Can't create database 'qa_database'; database exists'


In [21]:
query = """

CREATE TABLE qa_table2 (
    question TEXT,
    answer TEXT
);

"""
execute_query(connection, query)

Error: '1050 (42S01): Table 'qa_table2' already exists'


In [26]:
# Insert some common questions and answers into the database
insert_data_query = """

INSERT INTO qa_table (question, answer) VALUES 
(
    ("What is your name?", "My name is Basel."),
    ("What is the capital of France?", "PSG"),
    ("How old are you?", "I am an AI bot.")
);

"""

select_query = """SELECT * FROM qa_table;"""

execute_query(connection, select_query)

[('What is your name?', 'My name is Basel'),
 ('What is the capital of France?', 'Paris'),
 ('How old are you?', 'I am an AI bot')]

In [39]:
question = "What is your name?"
db_cursor = connection.cursor()
db_cursor.execute("SELECT answer FROM qa_table WHERE question = %s", (question,))
result = db_cursor.fetchone()
result[0]


'My name is Basel'

In [41]:
import re
import string


def preprocess_document(doc_bytes):
    """
    Args:
        :param doc_bytes: The document to be processed.

    Returns:
        :rtype: str
    """
    try:
        with open (doc_bytes, "r") as my_file:
            text = my_file.read()
            
            return text
        
    except:
        return False
    # Additional cleaning steps (optional)


# Main logic for answering questions
from transformers import pipeline
# Load the pre-trained question answering model
qa_model = pipeline("question-answering", model="timpal0l/mdeberta-v3-base-squad2")


def answer_question(doc_bytes, question):
    # clean_text = preprocess_document(doc_bytes)
    # If not found in the database, use the AI model to answer the question
    context = preprocess_document(doc_bytes)

    # Check if the question is in the database
    db_cursor = connection.cursor()
    db_cursor.execute("SELECT answer FROM qa_table WHERE question = %s", (question,))
    result = db_cursor.fetchone()
    if result:
        return result[0]  # If found in the database, return the answer
    

    # If not found in the database, use the AI model to answer the question
    answer = qa_model(question=question, context=context)["answer"]

    # Store the new question-answer pair in the database
    db_cursor.execute("INSERT INTO qa_table (question, answer) VALUES (%s, %s)", (question, answer))
    connection.commit()

    return answer


# User interface setup using Gradio

import gradio as gr

interface = gr.Interface(
    fn=answer_question,
    inputs=[gr.File(label="Upload Document"), "text"],
    outputs="text",
    title="Question Answering System",
)
interface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
